In [1]:
%pip install ultralytics opencv-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
# 📚 Bibliotheken importieren
import cv2
from ultralytics import solutions
import time


In [3]:
# 📍 Videoquelle und Modellpfad definieren
video_path = r"C:\Users\miche\Documents\GitHub\Chairlift_Gefahrenerkennung\5_Video\chairlift_vid1.mp4"
model_path = r"C:\Users\miche\Documents\GitHub\Chairlift_Gefahrenerkennung\6_CNN Model\best.pt"


### Nur Klasse 0 anzeigen

In [4]:
heatmap = solutions.Heatmap(
    model=model_path,
    show=True,
    colormap=cv2.COLORMAP_PARULA,
    classes=[0]  # Nur Klassen mit den Indizes 0 und 2 anzeigen
)



Ultralytics Solutions:  {'source': None, 'model': 'C:\\Users\\miche\\Documents\\GitHub\\Chairlift_Gefahrenerkennung\\6_CNN Model\\best.pt', 'classes': [0], 'show_conf': True, 'show_labels': True, 'region': None, 'colormap': 12, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'cropped-detections', 'json_file': None, 'line_width': 2, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': True, 'iou': 0.7, 'conf': 0.25, 'device': None, 'max_det': 300, 'half': False, 'tracker': 'botsort.yaml', 'verbose': True, 'data': 'images'}


### Alle Klassen anzeigen

In [5]:

# 🎨 Heatmap-Objekt initialisieren
heatmap = solutions.Heatmap(
    model=model_path,               # 🔍 Pfad zum trainierten YOLO11-Modell
    show=True,                      # 🖥️ Anzeige der Heatmap im Fenster
    colormap=cv2.COLORMAP_PARULA    # 🌈 Farbkarte für die Heatmap
)


Ultralytics Solutions:  {'source': None, 'model': 'C:\\Users\\miche\\Documents\\GitHub\\Chairlift_Gefahrenerkennung\\6_CNN Model\\best.pt', 'classes': None, 'show_conf': True, 'show_labels': True, 'region': None, 'colormap': 12, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'cropped-detections', 'json_file': None, 'line_width': 2, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': True, 'iou': 0.7, 'conf': 0.25, 'device': None, 'max_det': 300, 'half': False, 'tracker': 'botsort.yaml', 'verbose': True, 'data': 'images'}


In [ ]:
# 📹 Video öffnen
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), f"Fehler beim Öffnen des Videos: {video_path}"

# 🎞️ Videoeigenschaften abrufen
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 💾 Videoausgabe initialisieren
output_path = r"C:\Users\miche\Documents\GitHub\Chairlift_Gefahrenerkennung\5_Video\chairlift_vid1_heatmap_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 🔄 Video frameweise verarbeiten
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # 🧠 Heatmap und Bounding Boxes generieren
    results = heatmap(frame)

    # 💾 Frame mit Heatmap und Bounding Boxes speichern
    out.write(results.plot_im)

# 🧹 Ressourcen freigeben
cap.release()
out.release()
cv2.destroyAllWindows()

### 🔄 Alternative Optionen

Nur bestimmte Klassen visualisieren: Wenn du nur bestimmte Objektklassen in der Heatmap anzeigen möchtest, kannst du das classes-Argument verwenden. Beispiel:

In [ ]:
# Video öffnen
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Fehler beim Öffnen des Videos."

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Videoausgabe initialisieren
output_path = r"C:\Users\miche\Documents\GitHub\Chairlift_Gefahrenerkennung\5_Video\chairlift_vid1_heatmap_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_counter = 0
pause_duration = 2  # Sekunden pausieren

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame_counter += 1

    # Normale Inferenz mit Bounding Boxes
    results = heatmap(frame)
    normal_frame = results.plot_im

    # Schreibe normales Frame ins Video
    out.write(normal_frame)

    # Alle 30 Frames eine Heatmap-Pause einlegen
    if frame_counter % 25 == 0:
        print(f"Heatmap-Overlay bei Frame {frame_counter} angezeigt.")
        # Zeige das aktuelle Frame mit Heatmap für 2 Sekunden an
        cv2.imshow("Heatmap Pause", normal_frame)
        cv2.waitKey(int(pause_duration * 1000))  # Pause in ms

    # Normale Anzeige (optional)
    cv2.imshow("YOLO + Heatmap Video", normal_frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC zum Beenden
        break

# Ressourcen freigeben
cap.release()
out.release()
cv2.destroyAllWindows()


Ultralytics Solutions:  {'source': None, 'model': 'C:\\Users\\miche\\Documents\\GitHub\\Chairlift_Gefahrenerkennung\\6_CNN Model\\best.pt', 'classes': [0], 'show_conf': True, 'show_labels': True, 'region': None, 'colormap': 12, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'cropped-detections', 'json_file': None, 'line_width': 2, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': False, 'iou': 0.7, 'conf': 0.25, 'device': None, 'max_det': 300, 'half': False, 'tracker': 'botsort.yaml', 'verbose': True, 'data': 'images'}

0: 480x640 (no detections), 48.6ms
Speed: 3.8ms preprocess, 48.6ms inference, 30.9ms postprocess per image at shape (1, 3, 480, 640)
WARNING no tracks found!
 Results: SolutionResults()

0: 480x640 (no detections), 16.4ms
Speed: 3.7ms preprocess, 16.4ms inference, 1.4ms postproc

In [ ]:

# Video öffnen
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Fehler beim Öffnen des Videos."

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Videoausgabe initialisieren
output_path = r"C:\Users\miche\Documents\GitHub\Chairlift_Gefahrenerkennung\5_Video\chairlift_vid1_heatmap_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Heatmap und Bounding Boxes berechnen
    results = heatmap(frame)

    # Prüfen, ob Klasse 0 erkannt wurde
    has_class_0 = False
    if hasattr(results, 'pred') and results.pred and len(results.pred[0]) > 0:
        has_class_0 = any([int(d['class']) == 0 for d in results.pred[0]])

    # Wenn Klasse 0 erkannt: Frame 2 Sekunden anzeigen
    if has_class_0:
        frame_with_heatmap = results.plot_im

        # Heatmap-Frame 2 Sekunden anzeigen und ins Video schreiben
        start_time = time.time()
        while time.time() - start_time < 2:
            cv2.imshow("Heatmap Pause", frame_with_heatmap)
            out.write(frame_with_heatmap)
            if cv2.waitKey(30) & 0xFF == 27:  # ESC zum Abbrechen
                break
    else:
        frame_normal = results.plot_im
        cv2.imshow("YOLO + Heatmap Video", frame_normal)
        out.write(frame_normal)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# Ressourcen freigeben
cap.release()
out.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 49.8ms
Speed: 2.7ms preprocess, 49.8ms inference, 60.8ms postprocess per image at shape (1, 3, 480, 640)
WARNING no tracks found!
 Results: SolutionResults()

0: 480x640 (no detections), 12.5ms
Speed: 3.2ms preprocess, 12.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
WARNING no tracks found!
 Results: SolutionResults()

0: 480x640 (no detections), 12.5ms
Speed: 1.8ms preprocess, 12.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
WARNING no tracks found!
 Results: SolutionResults()

0: 480x640 (no detections), 9.9ms
Speed: 2.7ms preprocess, 9.9ms inference, 33.6ms postprocess per image at shape (1, 3, 480, 640)
WARNING no tracks found!
 Results: SolutionResults()

0: 480x640 (no detections), 12.0ms
Speed: 3.2ms preprocess, 12.0ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
WARNING no tracks found!
 Results: SolutionResults()

0: 480x640 (no detections), 10.5ms
Speed: 2.2ms preprocess, 10.5ms in